# Clustering

## Based on the article [PATS](http://ismir2002.ircam.fr/proceedings/OKPROC02-FP07-4.pdf)

The ideia is to establish a similarity metric and do a clusterization process with these metrics. 

In [1]:
# Importing libraries 
import pandas as pd 
import numpy as np 

from sklearn.cluster import AffinityPropagation, SpectralClustering, DBSCAN
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import cdist, squareform

from seaborn import heatmap 
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import glob
import os
import time

## Defining Important Features 

This features will be used to understand the data. 

In [62]:
metadata =  ['playlist_id', 'duration_ms', 'explicit', 'id', 'album_type', 'popularity', 'album_id', 
             'album_release_date', 'artists_ids', 'name', 'artists_names']
audio_features = ['danceability', 'energy', 'loudness', 'key', 'mode', 'speechiness', 'acousticness', 
                  'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature', 'id']

## Playlist and tracks dataframes 

Getting the data generated by Spotify API. 

In [63]:
playlists_df = pd.read_pickle('../data/sp_playlists.pkl')[['id']]
playlists_df['playlist_id'] = playlists_df['id']
playlists_df = playlists_df[['playlist_id']]
audio_features_df = pd.read_pickle('../data/sp_audio_features.pkl')[audio_features]
tracks_df = pd.concat(
    [pd.read_pickle(file)[metadata] for file in glob.glob('../data/sp_tracks_ready_*.pkl')],
    ignore_index=True
)

In [64]:
tracks_df = audio_features_df.merge(tracks_df, on = 'id')
tracks_df = tracks_df.merge(playlists_df, on = 'playlist_id')

In [65]:
del audio_features_df
del playlists_df 

I convert the dates to datetime and use the year as a continuum value. 

In [66]:
tracks_df['album_release_date'] = tracks_df['album_release_date'].apply(lambda x: 
                                                                        None if x == '0000' else pd.to_datetime(x))
tracks_df['years'] = tracks_df['album_release_date'].apply(lambda x: x.year + x.month/12 + x.day/365)
tracks_df.sample()

,danceability,energy,loudness,key,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,duration_ms,explicit,album_type,popularity,album_id,album_release_date,artists_ids,name,artists_names,years
49453,0.558,0.855,-5.243,2,1,0.0343,0.00342,0.934,0.175,0.574,...,340366.0,False,single,18.0,5O5GxUUPKUqrWvtgaFyZnM,2015-11-04,[4OrizGCKhOrW6iDDJHN9xd],Bent (Roi's Song),[DIIV],2015.927626


We have 45 nan values in the years columns. I will put the mean of the values, because it's few missing data.

In [67]:
tracks_df['years'].fillna(np.mean(tracks_df['years']), inplace = True)

I separate the catefortical, numerical and set_oriented features, to make the ideia of the similarity matrix. 

In [25]:
features_categorical =  ['explicit', 'album_type', 'album_id', 'key', 'mode', 'time_signature']
features_numerical = ['duration_ms', 'popularity', 'danceability', 'energy', 'loudness', 'speechiness', 
                      'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'years']
features_set_oriented = ['artists_ids']

features = [] 
features.extend(features_categorical)
features.extend(features_numerical)
features.extend(features_set_oriented)

Only to ensure correct type here.

In [68]:
tracks_df[features_numerical] = tracks_df[features_numerical].astype(float)

Let's build the metrics proposed. For now, I normalize the numerical data, ensuring the range to be $[0,1]$. 

In [69]:
scaler = MinMaxScaler()
tracks_df[features_numerical] = scaler.fit_transform(tracks_df[features_numerical])

In [70]:
metric_categorical = lambda x1,x2: x1 == x2
metric_set_oriented = lambda x1, x2: len(set(x1) & set(x2))/(max(len(x1), len(x2)))
metric_numerical = lambda x1, x2: 1 - abs(x1 - x2)

def metric_songs(x: np.array, y: np.array) -> float: 
    # Arbitrary choice 
    weight = np.ones(19)/19
    
    similarity = 0
    similarity += sum(weight[0:6]*metric_categorical(x[0:6], y[0:6]))
    similarity += sum(weight[6:18]*metric_numerical(x[6:18], y[6:18]))
    similarity += weight[18]*metric_set_oriented(x[18], y[18])

    return similarity

## Simple example :


Let's calculate a simple case with two songs.

In [54]:
x1 = np.array(tracks_df[features].iloc[10])
x2 = np.array(tracks_df[features].iloc[34])
metric_songs(x1, x2)

0.8309429212833058

## Calculate the Similarity Matrix

I use the pdist function to calculate distance between tracks. I sample only 2 thousand song, because it can take a long time to run the comparision between all songs. After I can convert this similarity matrix to a distance matrix if I wish. 

In [ ]:
#tracks_selected = tracks_df.sample(2000, random_state = 1000)
#tracks_similarity = pdist(tracks_selected[features].values, metric = metric_songs)

In [ ]:
# tracks_similarity = - lambda*np.log(lambda*tracks_similarity)
#tracks_similarity = squareform(tracks_similarity) + np.eye(2000)

In [ ]:
#fig, ax = plt.subplots(figsize = (15,12))
#heatmap(tracks_similarity, vmin = 0, vmax = tracks_similarity.max(), ax = ax)

#plt.show()

## Clusterization Algorithms

This algorithms require distance matrix. 

In [ ]:
#model = AffinityPropagation(affinity = 'precomputed').fit(tracks_similarity)

Quantos labels foram feitos? 

In [ ]:
#len(np.unique(model.labels_))

Quantas playlists realmente existiam?

In [ ]:
#len(tracks.iloc[0:2000].playlist_id.unique())

In [ ]:
#model2 = DBSCAN(eps = 0.25, metric = 'precomputed').fit(tracks_similarity)

In [ ]:
#len(model2.labels_[model2.labels_ == -1])

In [ ]:
#len(np.unique(model2.labels_))

In [ ]:
#model3 = SpectralClustering(n_clusters=27, affinity='precomputed').fit(tracks_similarity)

## Recommendation based on similarity. 

I use the metric specified like a probability. That probability tell us which factor we should recommend a song. In special, we choose, iteratively, the music that optimizes the product of probabilities, related to the likelihood. 

#### Test

I initialize with a random song and will take 10 songs to build a playlist.

In [58]:
number_of_songs = 10

In [141]:
# Functions

def get_similar_track(tracks_similarity: np.array) -> int: 
    
    interest_tracks = tracks_similarity.prod(axis = 0)
    song = np.argmax(interest_tracks)
    
    return song 

def get_playlist(number_of_songs: int, tracks: np.array, songs: list) -> list: 
    
    assert initial_song < tracks.shape[0]
    assert initial_song >= 0 
    
    n = len(songs)
    
    tracks_similarity = np.ones((number_of_songs + n, tracks.shape[0]))
    
    # calcugelate the similarity between track in the playlist
    tracks_similarity[0:n-1,:] = cdist(tracks[songs[:-1],:], tracks, metric = metric_songs)
    
    for s in range(n-1): 
        tracks_similarity[s,songs[s]] = 0
    
    for playlist_item in range(n-1, number_of_songs + n-1):

        tracks_similarity[playlist_item,:] = cdist(tracks[songs[len(songs)-1:],:], tracks, metric = metric_songs) 
        
        # Setting it to 0 take the used music to 0
        tracks_similarity[playlist_item, songs[-1]] = 0
        
        songs.append(get_similar_track(tracks_similarity))
    
    return songs

In [121]:
tracks_df_drop = tracks_df.drop_duplicates('id', ignore_index=True) 

In [101]:
playlist_test = tracks_df.sample()['playlist_id'].iloc[0]
playlist_tracks = tracks_df[tracks_df['playlist_id'] == playlist_test]

In [128]:
songs = list(tracks_df_drop[tracks_df_drop['id'].isin(tracks_df.loc[playlist_tracks.index]['id'])].index)

In [142]:
selected_songs = np.random.choice(songs, 50, replace = False)
playlist = get_playlist(20, tracks_df_drop[features].values, selected_songs)

KeyboardInterrupt: 

In [143]:
len(songs)

70